In [22]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.7

In [23]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [24]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)
bert_sc = bert_sc.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [25]:
text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。"
]
label_list = [1,0,1]

#符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items() }
labels = torch.tensor(label_list).cuda()

#推論
with torch.no_grad():
  output = bert_sc.forward(**encoding)
scores = output.logits
labels_predicted = scores.argmax(-1)
num_correct = (labels_predicted==labels).sum().item()
accuracy = num_correct/labels.size(0)

print("# scores:")
print(scores.size())
print("# predicted labels:")
print(labels_predicted)
print("# accuracy:")
print(accuracy)


# scores:
torch.Size([3, 2])
# predicted labels:
tensor([1, 0, 0], device='cuda:0')
# accuracy:
0.6666666666666666


In [26]:
#符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding['labels'] = torch.tensor(label_list)
encoding = { k: v.cuda() for k, v in encoding.items() }

#ロスの計算
output = bert_sc(**encoding)
loss = output.loss
loss

tensor(0.6747, device='cuda:0', grad_fn=<NllLossBackward>)

# livedoor news corpus

In [28]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 

!tar -zxf ldcc-20140209.tar.gz 

--2021-07-11 15:18:22--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.1’

ldcc-20140209.tar.g 100%[===================>]   8.44M  5.00MB/s    in 1.7s    

2021-07-11 15:18:25 (5.00 MB/s) - ‘ldcc-20140209.tar.gz.1’ saved [8855190/8855190]



1行目:記事のURL <br>
2行目：記事の作成日時 <br>
3行目:記事のタイトル <br>
4行目以降：記事の本文 <br>

In [29]:
!cat ./text/it-life-hack/it-life-hack-6342280.txt

http://news.livedoor.com/article/detail/6342280/
2012-03-06T13:00:00+0900
USB3.0対応で爆速データ転送！　9倍速のリーダー／ライター登場
USB3.0が登場してから今年で4年目となるがパソコン側でのUSB3.0ポート搭載が進んで来ても対応機器がなかなか充実していない現状がある。そんな中で新しく高速な読み取りが可能なメモリーカードリーダー／ライターが登場した。

バッファローコクヨサプライがUSB3.0対応のカードリーダー／ライターを発表した。SDHC対応のSD系メディアやコンパクトフラッシュ、メモリースティック系メディア、xDピクチャーカードといったデジカメやスマホ、携帯ゲームといった機器で使われている各種メディアを従来よりも短時間でPCに取り込むことが可能になる。

転送速度が5Gbps（理論値）とUSB2.0の480Mbpsと比べて爆速になったUSB3.0はPC側の対応が進んで来ていたが高速転送が生かせる周辺機器としては、外付けHDDや一部のUSBメモリーくらいしかなかった。これに多くのメディアが扱えるリーダー／ライターが加わることで手軽にUSB3.0の恩恵を受けることができるようになる。

今回発表されたのは、USB3.0ケーブルとカードリーダー本体が分かれるタイプの「BSCR09U3」シリーズ（3,240円）、USB3.0コネクタをカードリーダー本体に内蔵している「BSCRD04U3」シリーズ（2,690円）だ。共にホワイトとブラックのカラーバリエーションが用意される（発売は3月下旬以降）。

■リリースページ
■バッファローコクヨサプライ




■バッファローの記事をもっと見る
・約283gでカバンに入る！小型キーボードの驚くべき機能
・3種類のホットキーで使いやすい！AndroidとPCで使えるキーボードの魅力
・ドラえもんもビックリの新アイテム！マウスとキーボードが合体"OPAir"
・ありそうでなかった便利機能！ファイル仕分けする画期的なHDD


サンディスク SanDisk microSDHC 32GB（microSD 32GB） 超高速クラス4  変換アダプター付 世界国内シェアNo.1 バルク品
クチコミを見る


### データローダ化

In [30]:
dataset_for_loader = [
    {'data':torch.tensor([0,1]), 'labels':torch.tensor(0)},
    {'data':torch.tensor([2,3]), 'labels':torch.tensor(1)},
    {'data':torch.tensor([4,5]), 'labels':torch.tensor(2)},
    {'data':torch.tensor([6,7]), 'labels':torch.tensor(3)},
]
loader = DataLoader(dataset_for_loader, batch_size=2)

for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

# batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
# batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}


In [31]:
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)

for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

# batch 0
{'data': tensor([[0, 1],
        [6, 7]]), 'labels': tensor([0, 3])}
# batch 1
{'data': tensor([[2, 3],
        [4, 5]]), 'labels': tensor([1, 2])}


In [32]:
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

max_length = 128
dataset_for_loader=[]

for label, category in enumerate(tqdm(category_list)):
  for file in glob.glob('./text/{}/{}*'.format(category, category)):
    lines = open(file).read().splitlines()
    text = '_n'.join(lines[3:])
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
    )
    encoding['labels'] = label # ラベルを追加
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
    

100%|██████████| 9/9 [00:36<00:00,  4.09s/it]


In [33]:
print(dataset_for_loader[0])

{'input_ids': tensor([[    2, 19241,     5,  1428,     9,     6,  3330,   478,     5, 13365,
          5370,     5,  1278,    12,     6, 16809,   640,    14,  4590,    58,
            10,    53,    28,   707,     8, 18700,  2375,    23,   965,   573,
            35,   811,  1151,    24,    28,     6,    59,  7100,     8, 13365,
          5370,     5,  1278,   174,    13,    83, 28455,  4749,  4882,    10,
             5,     9,    36,   167,  9346,    38,     5,  6503,    14,  3731,
            10,    45,    75,    13,   625,     8,  1679,  1751,  1679,  1751,
            36,    73, 18328,  9294,     7,  1746,  1780,    49,  4065, 29648,
             5, 14458,  4692,    11,  9346,   895,  2739,    49,  1435,    11,
          2867,     5,    14,  3904,     5, 15591,     7,    58,    16,    21,
          2610,    38,    13,  4341,    16,  4831,    10, 18700,  2375,     8,
           167,  9346,    12,     6,  7656,    73,  2438,     5, 18178, 28454,
            26,     7, 15388,    10,  

In [44]:
random.shuffle(dataset_for_loader)

n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)

dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(
    dataset_val, batch_size=256
)
dataloader_test = DataLoader(
    dataset_test, batch_size=256
)

# Pytorch Lightning によるファインチューニング

In [48]:
from pytorch_lightning import BertForSequenceClassification_pl

In [46]:
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

#学習方法指定
trainer = pl.Trainer(
    gpus=1,
    max_epochs=10,
    callbacks=[checkpoint]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [49]:

model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)

trainer.fit(model, dataloader_train, dataloader_val)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

ValueError: ignored

In [ ]:
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./

In [ ]:
test = trainer.test(test_dataloaders=dataloader_test)

In [ ]:
#load
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
) 


model.bert_sc.save_pretrained('/model_transformers')

In [ ]:
bert_sc = BertForSequenceClassification.from_pretrained(
    '/model_transformers'
)